In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# save pipe.pkl to output data folder
# !cp <output-file-name> /content/drive/MyDrive/GitCode


In [ ]:
# !pip install table_evaluator
# !pip install ctgan
# !pip install synthcity
# !pip3 install torch torchvision torchaudio

In [ ]:
! git clone https://github.com/humayraR/fact-ai_Repl.git

In [4]:
import sys
sys.path.insert(0,'/content/fact-ai_Repl')

In [ ]:
%cd fact-ai_Repl

In [ ]:
%pwd

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, recall_score, roc_auc_score

from table_evaluator import TableEvaluator

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

import os.path, sys
from tests.utils import load_adult

import warnings

# synthcity absolute
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [8]:
log.add(sink=sys.stderr, level="INFO")
sys.path.append(os.getcwd())

In [9]:
import PreProcessData

clean_df = PreProcessData.clean_df('data/adult.data')
#arr_X, arr_y, adult_df = load_adult() # arr_X np array without label/target, arr_y np array of just label/target


In [10]:
loader = GenericDataLoader(
    clean_df,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)

In [11]:
# DAG and bias dicts for debiasing adult dataset
df_dag = [
    ('race', 'marital-status'), ('race', 'education'), ('race','occupation'), ('race','hours-per-week'), ('race','label'),

    ('age', 'marital-status'), ('age', 'education'), ('age','occupation'), ('age','hours-per-week'), ('age','workclass'), ('age','relationship'), ('age','label'),

    ('sex', 'marital-status'), ('sex', 'education'), ('sex','occupation'), ('sex','hours-per-week'), ('sex','workclass'), ('sex','relationship'), ('sex','label'),

    ('native-country', 'marital-status'), ('native-country', 'education'), ('native-country','hours-per-week'), ('native-country','workclass'), ('native-country','relationship'), ('native-country','label'),

    ('marital-status', 'education'), ('marital-status','occupation'), ('marital-status','hours-per-week'), ('marital-status','workclass'), ('marital-status','relationship'), ('marital-status','label'),

    ('education','occupation'), ('education','hours-per-week'), ('education','workclass'), ('education','relationship'), ('education','label'),

    ('occupation', 'label'),
    ('hours-per-week', 'label'),
    ('workclass', 'label'),
    ('relationship', 'label')
]

ftu = {'sex':['label']}

dp = {'sex':['label'],
      'marital-status':['label'],
      'relationship':['label'],
      'occupation':['label'],
      'hours-per-week':['label'],
      'workclass':['label'],
      'education':['label']
    }

cf = {'sex':['label'],
      'marital-status':['label'],
      'relationship':['label']
    }

In [9]:
#!pip install nvidia_cublas_cu11 #--yes

In [10]:
#!pip3 install torch torchvision torchaudio

In [ ]:
print(torch.version.cuda)

In [13]:
from synthcity.utils.serialization import load_from_file, save_to_file
from synthcity.utils.reproducibility import clear_cache, enable_reproducible_results
from pathlib import Path
from synthcity.plugins.privacy import plugin_decaf
torch.cuda.empty_cache()
clear_cache()

In [ ]:
# synthcity absolute
#from synthcity.plugins import Plugins
# save trained model


workspace = Path("DECAF_workspace")
generator_file = (workspace/"DECAFmodel_32b.bkp")

decaf_model = None
if generator_file.exists():
    decaf_model = load_from_file(generator_file)
else:
    decaf_model = plugin_decaf.DECAFPlugin(n_iter=20, n_iter_baseline=100, batch_size=32)
    decaf_model.fit(loader,dag=df_dag)
    save_to_file(generator_file, decaf_model)


In [19]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [12]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:5"

In [ ]:
decaf_ftu = decaf_model.generate(count=70000, biased_edges=ftu).dataframe()
decaf_ftu.head()

In [ ]:
decaf_ftu['label'].unique()

In [ ]:
def get_metrics(mode,df,synthetic):

    # Split the data into train,test
    traindf, testdf = train_test_split(df, test_size=0.3)
    X_train = traindf.loc[:, traindf.columns != 'label']
    y_train = traindf['label']
    X_test = testdf.loc[:, testdf.columns != 'label']
    y_test = testdf['label']

    clf_df = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',
                                     learning_rate='constant', learning_rate_init=0.001).fit(X_train, y_train)
    '''
    SYNTHETIC DATASET
    '''
    # Make sure the data is representative of the original dataset
    synthetic_balanced_1 = synthetic[synthetic.label == 1].sample(frac = 0.75)
    synthetic_balanced_0 = synthetic[synthetic.label == 0].sample(frac = 0.25)
    #synthetic_balanced = synthetic_balanced_1.append(synthetic_balanced_0)
    synthetic_balanced = pd.concat([synthetic_balanced_1, synthetic_balanced_0], ignore_index=True)

    # Split the data into train,test
    X_syn = synthetic_balanced.loc[:, synthetic_balanced.columns != 'label']
    y_syn = synthetic_balanced['label']

    y_pred_syn = clf_df.predict(X_syn)

    synthetic_pos = synthetic.assign(sex=0)
    synthetic_neg = synthetic.assign(sex=1)

    x_pos_syn = synthetic_balanced[synthetic_balanced['sex'] == 0].drop(['label'], axis = 1)[:7508]
    x_neg_syn = synthetic_balanced[synthetic_balanced['sex'] == 1].drop(['label'], axis = 1)[:7508]

    pos = clf_df.predict(synthetic_pos.drop('label',axis=1))
    neg = clf_df.predict(synthetic_neg.drop('label',axis=1))

    pred_pos_syn = clf_df.predict(x_pos_syn)
    pred_neg_syn = clf_df.predict(x_neg_syn)

    FTU = np.abs(np.mean(pos-neg))
    DP = np.mean(pred_pos_syn)-np.mean(pred_neg_syn)

    # Print the obtained statistics
    print('Statistics for dataset for mode:',mode)
    print('Precision:',precision_score(y_syn, y_pred_syn, average='binary'))
    print('Recall:',recall_score(y_syn, y_pred_syn, average='binary'))
    print('AUROC:',roc_auc_score(y_syn, y_pred_syn))
    print('FTU:',FTU)
    print('DP:',DP)

get_metrics("DECAF-FTU", decaf_ftu,clean_df)

In [ ]:
decaf_dp = decaf_model.generate(count=70000, biased_edges=dp).dataframe()
decaf_dp.head()

In [ ]:
decaf_dp['relationship'].unique()

In [ ]:
get_metrics("DECAF-DP", decaf_dp, clean_df)

In [ ]:
decaf_cf = decaf_model.generate(count=70000, biased_edges=cf).dataframe()
decaf_cf.head()

In [ ]:
decaf_cf['relationship'].unique()

In [ ]:
get_metrics("DECAF-cf", decaf_cf, clean_df)

In [38]:
#save csv

decaf_ftu.to_csv('decaf_ftu.csv', index=False)
decaf_dp.to_csv('decaf_dp.csv', index=False)
decaf_cf.to_csv('decaf_cf.csv', index=False)


In [ ]:
#get metrics for real data
get_metrics("Original Data",clean_df,clean_df)

In [ ]:
# Synthcity Benchmarking doesn't support DECAF at the moment. Can't pass in DAG or bias dict. Will need to build benchmarking from scratch for DECAF and other external GANs
# from SynthBenchmarks import Benchmarks

# score = Benchmarks.evaluate(
#     [("Test Decaf", "decaf", {"n_iter": 10, "n_iter_baseline": 100})],
#     loader,
#     synthetic_size=1000,
#     repeats=2
#     #synthetic_cache=False
#     #synthetic_reuse_if_exists=True
# )

In [40]:
clean_orig_df = PreProcessData.clean_df('data/adult.data')
traindf_orig_df, test_orig_df = train_test_split(clean_orig_df, test_size=0.95)

#synth data = ftu
ftu_loader = GenericDataLoader(
    decaf_ftu,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)


# test data = orig
#clean_df = PreProcessData.clean_df('data/adult.data')
test_loader = GenericDataLoader(
    test_orig_df,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)

In [42]:
from synthcity.plugins.privacy import plugin_dpgan

ftu_dpgan_e01 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=0.1)
ftu_dpgan_e01.fit(ftu_loader)

[2023-12-11T01:39:12.286232+0000][7296][INFO] Encoding race 8129583723855608205
[2023-12-11T01:39:12.304575+0000][7296][INFO] Encoding age 107970850829718555
[2023-12-11T01:39:18.063663+0000][7296][INFO] Encoding sex 5492054448805308787
[2023-12-11T01:39:18.086615+0000][7296][INFO] Encoding native-country 2320950422065707565
[2023-12-11T01:39:21.137016+0000][7296][INFO] Encoding marital-status 6745135823506738315
[2023-12-11T01:39:21.160683+0000][7296][INFO] Encoding education 2302411911603430902
[2023-12-11T01:39:22.663914+0000][7296][INFO] Encoding occupation 4932011153657206947
[2023-12-11T01:39:22.682931+0000][7296][INFO] Encoding hours-per-week 2495971846394436626
[2023-12-11T01:39:28.798286+0000][7296][INFO] Encoding workclass 3805186681709485554
[2023-12-11T01:39:28.817004+0000][7296][INFO] Encoding relationship 1569193566093787873
[2023-12-11T01:39:28.835122+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T01:39:32.148646+0000][7296][INFO] Training GAN on device

In [43]:
ftu_dpgan_e01_df = ftu_dpgan_e01.generate(count=50000).dataframe()
ftu_dpgan_e01_df.head()

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,3,20,1,17,5,14,12,34,4,1,1
1,1,22,1,17,5,14,0,34,4,3,1
2,1,17,1,19,5,14,0,34,4,3,1
3,4,22,1,17,5,14,11,34,4,3,1
4,3,22,1,17,5,14,3,34,4,3,1


In [44]:
get_metrics("ftu_dpgan_e01", ftu_dpgan_e01_df, clean_df)

Statistics for dataset for mode: ftu_dpgan_e01
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


In [45]:
from synthcity.benchmark import Benchmarks

ftu_dpgan_e01_score2 = Benchmarks.evaluate(
    [("dp_decaf_ftu_eps_0.1", "dpgan", {"epsilon": 0.1, "n_iter": 10, "n_iter_min": 1})],
    ftu_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(ftu_dpgan_e01_score2)

[2023-12-11T02:21:30.306789+0000][7296][INFO] Testcase : dp_decaf_ftu_eps_0.1
[2023-12-11T02:21:30.309850+0000][7296][INFO] [testcase] Experiment repeat: 0 task type: classification Train df hash = 6329755547212728994
[2023-12-11T02:21:34.265253+0000][7296][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T02:21:34.265253+0000][7296][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T02:21:34.265253+0000][7296][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T02:21:34.487291+0000][7296][INFO] Encoding race 139106487857773502
[2023-12-11T02:21:34.497908+0000][7296][INFO] Encoding age 6510218881667608903
[2023-12-11T02:21:38.144613+0000][7296][INFO] Encoding sex 7124804067471569966
[2023-12-11T02:21:38.159759+0000][7296][INFO] Encoding native-country 4826846401655929469
[2023


Plugin : dp_decaf_ftu_eps_0.1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.945627,0.945627,0.945627,0.0,0.945627,0.0,1,0,56.13
stats.prdc.recall,0.207720,0.207720,0.207720,0.0,0.207720,0.0,1,0,56.13
stats.prdc.density,0.537356,0.537356,0.537356,0.0,0.537356,0.0,1,0,56.13
stats.prdc.coverage,0.021254,0.021254,0.021254,0.0,0.021254,0.0,1,0,56.13
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.78
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.78
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.78
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,45.09
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,45.09
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,45.09


In [46]:
type(ftu_loader.dataframe()['label'][0])

numpy.int64

In [47]:
ftu_dpgan_e1 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=1)
ftu_dpgan_e1.fit(ftu_loader)
ftu_dpgan_e1_df = ftu_dpgan_e1.generate(count=50000).dataframe()
#ftu_dpgan_e1_df.head()

[2023-12-11T02:47:01.314189+0000][7296][INFO] Encoding race 8129583723855608205
[2023-12-11T02:47:01.326260+0000][7296][INFO] Encoding age 107970850829718555
[2023-12-11T02:47:05.286071+0000][7296][INFO] Encoding sex 5492054448805308787
[2023-12-11T02:47:05.302064+0000][7296][INFO] Encoding native-country 2320950422065707565
[2023-12-11T02:47:07.925797+0000][7296][INFO] Encoding marital-status 6745135823506738315
[2023-12-11T02:47:07.951441+0000][7296][INFO] Encoding education 2302411911603430902
[2023-12-11T02:47:09.422308+0000][7296][INFO] Encoding occupation 4932011153657206947
[2023-12-11T02:47:09.436912+0000][7296][INFO] Encoding hours-per-week 2495971846394436626
[2023-12-11T02:47:15.317409+0000][7296][INFO] Encoding workclass 3805186681709485554
[2023-12-11T02:47:15.329778+0000][7296][INFO] Encoding relationship 1569193566093787873
[2023-12-11T02:47:15.342807+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T02:47:18.053536+0000][7296][INFO] Training GAN on device

In [48]:
get_metrics("ftu_dpgan_e1", ftu_dpgan_e1_df, clean_df)
ftu_dpgan_e1_score2 = Benchmarks.evaluate(
    [("dp_decaf_ftu_eps_1", "dpgan", {"epsilon": 1, "n_iter": 10, "n_iter_min": 1})],
    ftu_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(ftu_dpgan_e1_score2)

[2023-12-11T03:21:14.930234+0000][7296][INFO] Testcase : dp_decaf_ftu_eps_1
[2023-12-11T03:21:14.933583+0000][7296][INFO] [testcase] Experiment repeat: 0 task type: classification Train df hash = 6329755547212728994
[2023-12-11T03:21:14.936621+0000][7296][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T03:21:14.936621+0000][7296][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T03:21:14.936621+0000][7296][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T03:21:15.052990+0000][7296][INFO] Encoding race 139106487857773502
[2023-12-11T03:21:15.064474+0000][7296][INFO] Encoding age 6510218881667608903


Statistics for dataset for mode: ftu_dpgan_e1
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T03:21:18.289774+0000][7296][INFO] Encoding sex 7124804067471569966
[2023-12-11T03:21:18.302635+0000][7296][INFO] Encoding native-country 4826846401655929469
[2023-12-11T03:21:22.130020+0000][7296][INFO] Encoding marital-status 7839442428620044333
[2023-12-11T03:21:22.145339+0000][7296][INFO] Encoding education 2480090733242301894
[2023-12-11T03:21:24.394151+0000][7296][INFO] Encoding occupation 3214762354189394834
[2023-12-11T03:21:24.406082+0000][7296][INFO] Encoding hours-per-week 1036015838165215767
[2023-12-11T03:21:27.420421+0000][7296][INFO] Encoding workclass 7552893260435030203
[2023-12-11T03:21:27.432616+0000][7296][INFO] Encoding relationship 3406607818822282356
[2023-12-11T03:21:27.444621+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T03:21:29.521345+0000][7296][INFO] Training GAN on device cpu. features = 55
100%|██████████| 10/10 [17:20<00:00, 104.08s/it]
[2023-12-11T03:39:14.526001+0000][7296][ERROR]  >> Evaluator stats.wasserstein_dist faile


Plugin : dp_decaf_ftu_eps_1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.766071,0.766071,0.766071,0.0,0.766071,0.0,1,0,58.56
stats.prdc.recall,0.548335,0.548335,0.548335,0.0,0.548335,0.0,1,0,58.56
stats.prdc.density,0.444678,0.444678,0.444678,0.0,0.444678,0.0,1,0,58.56
stats.prdc.coverage,0.019962,0.019962,0.019962,0.0,0.019962,0.0,1,0,58.56
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.87
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.87
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.87
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,44.40
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,44.40
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,44.40


In [49]:
ftu_dpgan_e10 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=10)
ftu_dpgan_e10.fit(ftu_loader)
ftu_dpgan_e10_df = ftu_dpgan_e10.generate(count=50000).dataframe()
# ftu_dpgan_e10_df.head()
get_metrics("ftu_dpgan_e10", ftu_dpgan_e10_df, clean_df)
ftu_dpgan_e10_score2 = Benchmarks.evaluate(
    [("dp_decaf_ftu_eps_10", "dpgan", {"epsilon": 10, "n_iter": 10, "n_iter_min": 1})],
    ftu_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(ftu_dpgan_e10_score2)

[2023-12-11T03:45:27.532482+0000][7296][INFO] Encoding race 8129583723855608205
[2023-12-11T03:45:27.543739+0000][7296][INFO] Encoding age 107970850829718555
[2023-12-11T03:45:31.996820+0000][7296][INFO] Encoding sex 5492054448805308787
[2023-12-11T03:45:32.011018+0000][7296][INFO] Encoding native-country 2320950422065707565
[2023-12-11T03:45:34.448314+0000][7296][INFO] Encoding marital-status 6745135823506738315
[2023-12-11T03:45:34.466386+0000][7296][INFO] Encoding education 2302411911603430902
[2023-12-11T03:45:35.776337+0000][7296][INFO] Encoding occupation 4932011153657206947
[2023-12-11T03:45:35.789386+0000][7296][INFO] Encoding hours-per-week 2495971846394436626
[2023-12-11T03:45:40.537734+0000][7296][INFO] Encoding workclass 3805186681709485554
[2023-12-11T03:45:40.550449+0000][7296][INFO] Encoding relationship 1569193566093787873
[2023-12-11T03:45:40.563636+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T03:45:43.414154+0000][7296][INFO] Training GAN on device

Statistics for dataset for mode: ftu_dpgan_e10
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T04:12:21.402077+0000][7296][INFO] Encoding sex 7124804067471569966
[2023-12-11T04:12:21.414165+0000][7296][INFO] Encoding native-country 4826846401655929469
[2023-12-11T04:12:25.285016+0000][7296][INFO] Encoding marital-status 7839442428620044333
[2023-12-11T04:12:25.299615+0000][7296][INFO] Encoding education 2480090733242301894
[2023-12-11T04:12:27.699670+0000][7296][INFO] Encoding occupation 3214762354189394834
[2023-12-11T04:12:27.711470+0000][7296][INFO] Encoding hours-per-week 1036015838165215767
[2023-12-11T04:12:30.644827+0000][7296][INFO] Encoding workclass 7552893260435030203
[2023-12-11T04:12:30.656294+0000][7296][INFO] Encoding relationship 3406607818822282356
[2023-12-11T04:12:30.668902+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T04:12:32.782034+0000][7296][INFO] Training GAN on device cpu. features = 55
100%|██████████| 10/10 [18:32<00:00, 111.25s/it]
[2023-12-11T04:31:32.623799+0000][7296][ERROR]  >> Evaluator stats.wasserstein_dist faile


Plugin : dp_decaf_ftu_eps_10


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.889963,0.889963,0.889963,0.0,0.889963,0.0,1,0,59.26
stats.prdc.recall,0.080094,0.080094,0.080094,0.0,0.080094,0.0,1,0,59.26
stats.prdc.density,0.503630,0.503630,0.503630,0.0,0.503630,0.0,1,0,59.26
stats.prdc.coverage,0.030641,0.030641,0.030641,0.0,0.030641,0.0,1,0,59.26
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.75
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.75
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.75
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,43.38
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,43.38
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,43.38


In [50]:
#DECAF_DP_DP
dp_loader = GenericDataLoader(
    decaf_dp,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)
dp_dpgan_e01 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=0.1)
dp_dpgan_e01.fit(dp_loader)
dp_dpgan_e01_df = dp_dpgan_e01.generate(count=50000).dataframe()
#dp_dpgan_e01_df.head()
get_metrics("dp_dpgan_e01", dp_dpgan_e01_df, clean_df)
dp_dpgan_e01_score2 = Benchmarks.evaluate(
    [("dp_decaf_dp_eps_01", "dpgan", {"epsilon": 0.1, "n_iter": 10, "n_iter_min": 1})],
    dp_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(dp_dpgan_e01_score2)

[2023-12-11T04:52:00.554195+0000][7296][INFO] Encoding race 7991478587696464017
[2023-12-11T04:52:00.565238+0000][7296][INFO] Encoding age 8254533536052928808
[2023-12-11T04:52:05.202655+0000][7296][INFO] Encoding sex 6502527295603552001
[2023-12-11T04:52:05.217459+0000][7296][INFO] Encoding native-country 5550277982148581195
[2023-12-11T04:52:08.111997+0000][7296][INFO] Encoding marital-status 8739976636599837275
[2023-12-11T04:52:08.128850+0000][7296][INFO] Encoding education 737507603031952893
[2023-12-11T04:52:10.176269+0000][7296][INFO] Encoding occupation 2687475736724702278
[2023-12-11T04:52:10.190200+0000][7296][INFO] Encoding hours-per-week 4859868023258817127
[2023-12-11T04:52:14.298646+0000][7296][INFO] Encoding workclass 5651321456396621178
[2023-12-11T04:52:14.313155+0000][7296][INFO] Encoding relationship 58963289932447824
[2023-12-11T04:52:14.325699+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T04:52:17.238060+0000][7296][INFO] Training GAN on device c

Statistics for dataset for mode: dp_dpgan_e01
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T05:12:11.771448+0000][7296][INFO] Encoding sex 3443807721064180858
[2023-12-11T05:12:11.785646+0000][7296][INFO] Encoding native-country 6010598154701329806
[2023-12-11T05:12:14.811315+0000][7296][INFO] Encoding marital-status 100538418146867491
[2023-12-11T05:12:14.831811+0000][7296][INFO] Encoding education 6947805141173533591
[2023-12-11T05:12:16.659425+0000][7296][INFO] Encoding occupation 53993213664906153
[2023-12-11T05:12:16.673047+0000][7296][INFO] Encoding hours-per-week 5285652187839956495
[2023-12-11T05:12:19.637132+0000][7296][INFO] Encoding workclass 5600446446339060301
[2023-12-11T05:12:19.650283+0000][7296][INFO] Encoding relationship 8353906456438579496
[2023-12-11T05:12:19.662566+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T05:12:21.749540+0000][7296][INFO] Training GAN on device cpu. features = 55
100%|██████████| 10/10 [16:35<00:00, 99.55s/it]
[2023-12-11T05:29:10.634627+0000][7296][ERROR]  >> Evaluator stats.wasserstein_dist failed: n


Plugin : dp_decaf_dp_eps_01


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.944720,0.944720,0.944720,0.0,0.944720,0.0,1,0,58.72
stats.prdc.recall,0.355867,0.355867,0.355867,0.0,0.355867,0.0,1,0,58.72
stats.prdc.density,0.465813,0.465813,0.465813,0.0,0.465813,0.0,1,0,58.72
stats.prdc.coverage,0.020137,0.020137,0.020137,0.0,0.020137,0.0,1,0,58.72
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.77
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.77
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.77
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,45.92
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,45.92
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,45.92


In [51]:
dp_dpgan_e1 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=1)
dp_dpgan_e1.fit(dp_loader)
dp_dpgan_e1_df = dp_dpgan_e1.generate(count=50000).dataframe()
#dp_dpgan_e1_df.head()
get_metrics("dp_dpgan_e1", dp_dpgan_e1_df, clean_df)
dp_dpgan_e1_score2 = Benchmarks.evaluate(
    [("dp_decaf_dp_eps_1", "dpgan", {"epsilon": 1, "n_iter": 10, "n_iter_min": 1})],
    dp_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(dp_dpgan_e1_score2)

[2023-12-11T06:02:54.923700+0000][7296][INFO] Encoding race 7991478587696464017
[2023-12-11T06:02:54.936331+0000][7296][INFO] Encoding age 8254533536052928808
[2023-12-11T06:02:59.757789+0000][7296][INFO] Encoding sex 6502527295603552001
[2023-12-11T06:02:59.773990+0000][7296][INFO] Encoding native-country 5550277982148581195
[2023-12-11T06:03:02.400590+0000][7296][INFO] Encoding marital-status 8739976636599837275
[2023-12-11T06:03:02.418185+0000][7296][INFO] Encoding education 737507603031952893
[2023-12-11T06:03:04.712828+0000][7296][INFO] Encoding occupation 2687475736724702278
[2023-12-11T06:03:04.729382+0000][7296][INFO] Encoding hours-per-week 4859868023258817127
[2023-12-11T06:03:09.283436+0000][7296][INFO] Encoding workclass 5651321456396621178
[2023-12-11T06:03:09.300376+0000][7296][INFO] Encoding relationship 58963289932447824
[2023-12-11T06:03:09.316465+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T06:03:12.441316+0000][7296][INFO] Training GAN on device c

Statistics for dataset for mode: dp_dpgan_e1
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T06:25:02.174416+0000][7296][INFO] Encoding sex 3443807721064180858
[2023-12-11T06:25:02.192525+0000][7296][INFO] Encoding native-country 6010598154701329806
[2023-12-11T06:25:04.600086+0000][7296][INFO] Encoding marital-status 100538418146867491
[2023-12-11T06:25:04.617912+0000][7296][INFO] Encoding education 6947805141173533591
[2023-12-11T06:25:06.383627+0000][7296][INFO] Encoding occupation 53993213664906153
[2023-12-11T06:25:06.398397+0000][7296][INFO] Encoding hours-per-week 5285652187839956495
[2023-12-11T06:25:09.255811+0000][7296][INFO] Encoding workclass 5600446446339060301
[2023-12-11T06:25:09.269954+0000][7296][INFO] Encoding relationship 8353906456438579496
[2023-12-11T06:25:09.282512+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T06:25:11.443171+0000][7296][INFO] Training GAN on device cpu. features = 55
100%|██████████| 10/10 [15:58<00:00, 95.87s/it]
[2023-12-11T06:41:29.349479+0000][7296][ERROR]  >> Evaluator stats.wasserstein_dist failed: n


Plugin : dp_decaf_dp_eps_1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.640469,0.640469,0.640469,0.0,0.640469,0.0,1,0,59.85
stats.prdc.recall,0.098450,0.098450,0.098450,0.0,0.098450,0.0,1,0,59.85
stats.prdc.density,0.267383,0.267383,0.267383,0.0,0.267383,0.0,1,0,59.85
stats.prdc.coverage,0.031898,0.031898,0.031898,0.0,0.031898,0.0,1,0,59.85
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.97
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.97
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.97
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,45.52
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,45.52
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,45.52


In [52]:
dp_dpgan_e10 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=10)
dp_dpgan_e10.fit(dp_loader)
dp_dpgan_e10_df = dp_dpgan_e10.generate(count=50000).dataframe()
#dp_dpgan_e1_df.head()
get_metrics("dp_dpgan_e10", dp_dpgan_e1_df, clean_df)
dp_dpgan_e10_score2 = Benchmarks.evaluate(
    [("dp_decaf_dp_eps_10", "dpgan", {"epsilon": 10, "n_iter": 10, "n_iter_min": 1})],
    dp_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(dp_dpgan_e10_score2)

[2023-12-11T08:12:28.109408+0000][7296][INFO] Encoding race 7991478587696464017
[2023-12-11T08:12:28.140420+0000][7296][INFO] Encoding age 8254533536052928808
[2023-12-11T08:12:33.104520+0000][7296][INFO] Encoding sex 6502527295603552001
[2023-12-11T08:12:33.120293+0000][7296][INFO] Encoding native-country 5550277982148581195
[2023-12-11T08:12:36.018143+0000][7296][INFO] Encoding marital-status 8739976636599837275
[2023-12-11T08:12:36.034717+0000][7296][INFO] Encoding education 737507603031952893
[2023-12-11T08:12:38.471133+0000][7296][INFO] Encoding occupation 2687475736724702278
[2023-12-11T08:12:38.486513+0000][7296][INFO] Encoding hours-per-week 4859868023258817127
[2023-12-11T08:12:43.198269+0000][7296][INFO] Encoding workclass 5651321456396621178
[2023-12-11T08:12:43.214516+0000][7296][INFO] Encoding relationship 58963289932447824
[2023-12-11T08:12:43.228852+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T08:12:46.687086+0000][7296][INFO] Training GAN on device c

Statistics for dataset for mode: dp_dpgan_e10
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T08:35:54.456654+0000][7296][INFO] Encoding race 296759257147630512
[2023-12-11T08:35:54.468141+0000][7296][INFO] Encoding age 7662042398102993154
[2023-12-11T08:35:58.552705+0000][7296][INFO] Encoding sex 3443807721064180858
[2023-12-11T08:35:58.566802+0000][7296][INFO] Encoding native-country 6010598154701329806
[2023-12-11T08:36:00.932251+0000][7296][INFO] Encoding marital-status 100538418146867491
[2023-12-11T08:36:00.949671+0000][7296][INFO] Encoding education 6947805141173533591
[2023-12-11T08:36:02.846951+0000][7296][INFO] Encoding occupation 53993213664906153
[2023-12-11T08:36:02.860785+0000][7296][INFO] Encoding hours-per-week 5285652187839956495
[2023-12-11T08:36:06.024391+0000][7296][INFO] Encoding workclass 5600446446339060301
[2023-12-11T08:36:06.037462+0000][7296][INFO] Encoding relationship 8353906456438579496
[2023-12-11T08:36:06.050093+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T08:36:08.230173+0000][7296][INFO] Training GAN on device cp


Plugin : dp_decaf_dp_eps_10


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.951769,0.951769,0.951769,0.0,0.951769,0.0,1,0,65.66
stats.prdc.recall,0.432714,0.432714,0.432714,0.0,0.432714,0.0,1,0,65.66
stats.prdc.density,0.802457,0.802457,0.802457,0.0,0.802457,0.0,1,0,65.66
stats.prdc.coverage,0.039471,0.039471,0.039471,0.0,0.039471,0.0,1,0,65.66
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.99
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.99
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.99
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,46.78
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,46.78
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,46.78


In [53]:
# CF DP GAN
cf_loader = GenericDataLoader(
    decaf_cf,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)
cf_dpgan_e01 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=0.1)
cf_dpgan_e01.fit(cf_loader)
cf_dpgan_e01_df = cf_dpgan_e01.generate(count=50000).dataframe()
#dp_dpgan_e01_df.head()
get_metrics("cf_dpgan_e01", cf_dpgan_e01_df, clean_df)
cf_dpgan_e01_score2 = Benchmarks.evaluate(
    [("cf_decaf_dp_eps_01", "dpgan", {"epsilon": 0.1, "n_iter": 10, "n_iter_min": 1})],
    cf_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(cf_dpgan_e01_score2)

[2023-12-11T08:59:36.606269+0000][7296][INFO] Encoding race 1383017471984994016
[2023-12-11T08:59:36.620369+0000][7296][INFO] Encoding age 6289251998071545774
[2023-12-11T08:59:40.838468+0000][7296][INFO] Encoding sex 6077034570685462839
[2023-12-11T08:59:40.856603+0000][7296][INFO] Encoding native-country 6662157537451627226
[2023-12-11T08:59:43.625485+0000][7296][INFO] Encoding marital-status 6922518090223717949
[2023-12-11T08:59:43.642339+0000][7296][INFO] Encoding education 1401733639004807383
[2023-12-11T08:59:46.694202+0000][7296][INFO] Encoding occupation 5264653642836518127
[2023-12-11T08:59:46.740225+0000][7296][INFO] Encoding hours-per-week 1314312012770718894
[2023-12-11T08:59:51.559494+0000][7296][INFO] Encoding workclass 5932589423120692341
[2023-12-11T08:59:51.575059+0000][7296][INFO] Encoding relationship 1324589051830165423
[2023-12-11T08:59:51.589425+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T08:59:54.709817+0000][7296][INFO] Training GAN on devic

Statistics for dataset for mode: cf_dpgan_e01
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T09:23:22.495242+0000][7296][INFO] Encoding age 4926583244563703893
[2023-12-11T09:23:25.984054+0000][7296][INFO] Encoding sex 1884990230754487006
[2023-12-11T09:23:25.996878+0000][7296][INFO] Encoding native-country 1100341709083395224
[2023-12-11T09:23:28.166581+0000][7296][INFO] Encoding marital-status 7953337587194734961
[2023-12-11T09:23:28.185403+0000][7296][INFO] Encoding education 3655031515746113582
[2023-12-11T09:23:29.650051+0000][7296][INFO] Encoding occupation 2323232729807037308
[2023-12-11T09:23:29.666009+0000][7296][INFO] Encoding hours-per-week 2523335744235026323
[2023-12-11T09:23:33.807735+0000][7296][INFO] Encoding workclass 8268731579842166593
[2023-12-11T09:23:33.819976+0000][7296][INFO] Encoding relationship 6302536925489205997
[2023-12-11T09:23:33.831697+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T09:23:36.042304+0000][7296][INFO] Training GAN on device cpu. features = 55
100%|██████████| 10/10 [18:52<00:00, 113.22s/it]
[2023-12-1


Plugin : cf_decaf_dp_eps_01


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.805158,0.805158,0.805158,0.0,0.805158,0.0,1,0,60.14
stats.prdc.recall,0.335102,0.335102,0.335102,0.0,0.335102,0.0,1,0,60.14
stats.prdc.density,0.482872,0.482872,0.482872,0.0,0.482872,0.0,1,0,60.14
stats.prdc.coverage,0.038075,0.038075,0.038075,0.0,0.038075,0.0,1,0,60.14
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.85
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.85
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.85
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,47.55
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,47.55
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,47.55


In [54]:
cf_dpgan_e1 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=1)
cf_dpgan_e1.fit(cf_loader)
cf_dpgan_e1_df = cf_dpgan_e1.generate(count=50000).dataframe()
#dp_dpgan_e01_df.head()
get_metrics("cf_dpgan_e1", cf_dpgan_e1_df, clean_df)
cf_dpgan_e1_score2 = Benchmarks.evaluate(
    [("cf_decaf_dp_eps_1", "dpgan", {"epsilon": 1, "n_iter": 10, "n_iter_min": 1})],
    cf_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(cf_dpgan_e1_score2)

[2023-12-11T09:48:45.947575+0000][7296][INFO] Encoding race 1383017471984994016
[2023-12-11T09:48:45.960779+0000][7296][INFO] Encoding age 6289251998071545774
[2023-12-11T09:48:50.414667+0000][7296][INFO] Encoding sex 6077034570685462839
[2023-12-11T09:48:50.432891+0000][7296][INFO] Encoding native-country 6662157537451627226
[2023-12-11T09:48:52.883307+0000][7296][INFO] Encoding marital-status 6922518090223717949
[2023-12-11T09:48:52.899627+0000][7296][INFO] Encoding education 1401733639004807383
[2023-12-11T09:48:55.692307+0000][7296][INFO] Encoding occupation 5264653642836518127
[2023-12-11T09:48:55.707558+0000][7296][INFO] Encoding hours-per-week 1314312012770718894
[2023-12-11T09:49:00.780224+0000][7296][INFO] Encoding workclass 5932589423120692341
[2023-12-11T09:49:00.793917+0000][7296][INFO] Encoding relationship 1324589051830165423
[2023-12-11T09:49:00.807303+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T09:49:03.800000+0000][7296][INFO] Training GAN on devic

Statistics for dataset for mode: cf_dpgan_e1
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T10:14:07.540807+0000][7296][INFO] Encoding race 5121713015216186991
[2023-12-11T10:14:07.554091+0000][7296][INFO] Encoding age 4926583244563703893
[2023-12-11T10:14:11.433907+0000][7296][INFO] Encoding sex 1884990230754487006
[2023-12-11T10:14:11.447687+0000][7296][INFO] Encoding native-country 1100341709083395224
[2023-12-11T10:14:13.808486+0000][7296][INFO] Encoding marital-status 7953337587194734961
[2023-12-11T10:14:13.831018+0000][7296][INFO] Encoding education 3655031515746113582
[2023-12-11T10:14:15.308245+0000][7296][INFO] Encoding occupation 2323232729807037308
[2023-12-11T10:14:15.322460+0000][7296][INFO] Encoding hours-per-week 2523335744235026323
[2023-12-11T10:14:19.629084+0000][7296][INFO] Encoding workclass 8268731579842166593
[2023-12-11T10:14:19.641649+0000][7296][INFO] Encoding relationship 6302536925489205997
[2023-12-11T10:14:19.654081+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T10:14:21.842562+0000][7296][INFO] Training GAN on devic


Plugin : cf_decaf_dp_eps_1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.856250,0.856250,0.856250,0.0,0.856250,0.0,1,0,62.43
stats.prdc.recall,0.781636,0.781636,0.781636,0.0,0.781636,0.0,1,0,62.43
stats.prdc.density,0.598234,0.598234,0.598234,0.0,0.598234,0.0,1,0,62.43
stats.prdc.coverage,0.016752,0.016752,0.016752,0.0,0.016752,0.0,1,0,62.43
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,0.85
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.85
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,0.85
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,46.98
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,46.98
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,46.98


In [55]:
cf_dpgan_e10 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=10)
cf_dpgan_e10.fit(cf_loader)
cf_dpgan_e10_df = cf_dpgan_e10.generate(count=50000).dataframe()
#dp_dpgan_e01_df.head()
get_metrics("cf_dpgan_e10", cf_dpgan_e10_df, clean_df)
cf_dpgan_e10_score2 = Benchmarks.evaluate(
    [("cf_decaf_dp_eps_10", "dpgan", {"epsilon": 10, "n_iter": 10, "n_iter_min": 1})],
    cf_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(cf_dpgan_e10_score2)

[2023-12-11T10:59:17.648359+0000][7296][INFO] Encoding race 1383017471984994016
[2023-12-11T10:59:17.661650+0000][7296][INFO] Encoding age 6289251998071545774
[2023-12-11T10:59:22.153561+0000][7296][INFO] Encoding sex 6077034570685462839
[2023-12-11T10:59:22.170877+0000][7296][INFO] Encoding native-country 6662157537451627226
[2023-12-11T10:59:24.694968+0000][7296][INFO] Encoding marital-status 6922518090223717949
[2023-12-11T10:59:24.714547+0000][7296][INFO] Encoding education 1401733639004807383
[2023-12-11T10:59:27.791404+0000][7296][INFO] Encoding occupation 5264653642836518127
[2023-12-11T10:59:27.805573+0000][7296][INFO] Encoding hours-per-week 1314312012770718894
[2023-12-11T10:59:32.649322+0000][7296][INFO] Encoding workclass 5932589423120692341
[2023-12-11T10:59:32.664543+0000][7296][INFO] Encoding relationship 1324589051830165423
[2023-12-11T10:59:32.677892+0000][7296][INFO] Encoding label 7226906040306014213
[2023-12-11T10:59:35.754210+0000][7296][INFO] Training GAN on devic

Statistics for dataset for mode: cf_dpgan_e10
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-11T11:22:06.559116+0000][7296][INFO] Encoding sex 1884990230754487006
[2023-12-11T11:22:06.572148+0000][7296][INFO] Encoding native-country 1100341709083395224
[2023-12-11T11:22:08.660953+0000][7296][INFO] Encoding marital-status 7953337587194734961
[2023-12-11T11:22:08.674411+0000][7296][INFO] Encoding education 3655031515746113582
[2023-12-11T11:22:10.270150+0000][7296][INFO] Encoding occupation 2323232729807037308
[2023-12-11T11:22:10.282212+0000][7296][INFO] Encoding hours-per-week 2523335744235026323
[2023-12-11T11:22:14.631249+0000][7296][INFO] Encoding workclass 8268731579842166593
[2023-12-11T11:22:14.643333+0000][7296][INFO] Encoding relationship 6302536925489205997
[2023-12-11T11:22:14.655730+0000][7296][INFO] Encoding label 7281877406956818018
[2023-12-11T11:22:16.668398+0000][7296][INFO] Training GAN on device cpu. features = 55
100%|██████████| 10/10 [17:04<00:00, 102.42s/it]
[2023-12-11T11:39:49.047010+0000][7296][ERROR]  >> Evaluator stats.wasserstein_dist faile


Plugin : cf_decaf_dp_eps_10


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.798039,0.798039,0.798039,0.0,0.798039,0.0,1,0,62.58
stats.prdc.recall,0.565959,0.565959,0.565959,0.0,0.565959,0.0,1,0,62.58
stats.prdc.density,0.406770,0.406770,0.406770,0.0,0.406770,0.0,1,0,62.58
stats.prdc.coverage,0.038075,0.038075,0.038075,0.0,0.038075,0.0,1,0,62.58
performance.linear_model.gt,0.813562,0.813562,0.813562,0.0,0.813562,0.0,1,0,1.06
performance.linear_model.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,1.06
performance.linear_model.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,1.06
performance.mlp.gt,0.499449,0.499449,0.499449,0.0,0.499449,0.0,1,0,48.23
performance.mlp.syn_id,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,48.23
performance.mlp.syn_ood,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1,0,48.23
